
# Section 2 — Dense Vector Retrieval (avec Hugging Face)

Ce notebook illustre la **récupération dense** : on représente requêtes et documents par des **vecteurs continus** (*embeddings*), puis on cherche les **voisins les plus proches**.  
L'objectif est de capturer le **sens** (sémantique) au-delà du simple recouvrement de termes.

**Plan :**
1. **Génération d'embeddings**  
   - Rappel rapide : *Word2Vec*, *GloVe* (mots) → moyennage pour phrases.  
   - Embeddings de **phrases** avec des modèles *Hugging Face* (*Sentence-Transformers*).
2. **Indexation & Recherche** (k-NN, FAISS).  
3. **Re-ranking** avec un **Cross-Encoder** (HF) pour améliorer l'ordre des résultats.



## Pré-requis (installation)
Exécuter la cellule suivante **une seule fois** pour installer les dépendances si nécessaire.  
> Si vous êtes hors-ligne, laissez-la de côté et lisez simplement le code/exemples.


In [ ]:

# Optionnel : décommentez pour installer
# %pip -q install --upgrade pip
# %pip -q install sentence-transformers faiss-cpu gensim torch --extra-index-url https://download.pytorch.org/whl/cpu



## Données d'exemple
On réutilise un mini-corpus multi-lingue proche de la Section 1 et quelques requêtes.


In [2]:

from typing import List, Tuple
import numpy as np

# Un mini-corpus jouet (FR + un peu d'EN pour montrer la robustesse)
corpus = [
    # --- Concepts généraux & définitions ---
    "La recherche d'information (IR) vise à retrouver des documents pertinents pour un besoin utilisateur exprimé sous forme de requête.",
    "Un pipeline RI comporte souvent : prétraitement, indexation, pondération, scoring, puis évaluation des résultats.",
    "Le modèle sac de mots ignore l'ordre des termes et représente un document par la fréquence des mots.",
    "TF-IDF pondère fort les mots fréquents dans un document mais rares dans le corpus ; TF peut être brut, log-normalisé ou binaire.",
    "Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.",
    "BM25+ et BM25L sont des variantes qui ajustent la normalisation de longueur et l'offset des faibles fréquences.",
    "Les embeddings denses encodent le sens dans un espace vectoriel continu, utiles pour la recherche sémantique.",
    "La recherche vectorielle peut être clairsemée (sparse) ou dense selon la représentation.",
    "RAG (Retrieval-Augmented Generation) combine récupération de passages et modèles génératifs pour produire des réponses sourcées.",
    "La pertinence peut être binaire ou graduée (gains), ce qui change la métrique optimale.",

    # --- Prétraitement & normalisation ---
    "La normalisation inclut les accents, la casse, la tokenisation, le stemming et la lemmatisation.",
    "Les mots vides (stopwords) comme 'le', 'la', 'de' sont souvent supprimés pour réduire le bruit.",
    "La lemmatisation française nécessite de gérer les accords et les formes verbales composées.",
    "Le stemming de Porter tronque les suffixes, au risque d'ambiguïtés (ex. 'nation' et 'national').",
    "Les fautes de frappe et variantes orthographiques (ex. 'recherhe', 'recherche') pénalisent les méthodes exactes.",
    "Les entités nommées (NER) aident à conserver des unités sémantiques comme 'OpenAI' ou 'Europe/Paris'.",

    # --- Outils & bibliothèques ---
    "Scikit-learn fournit CountVectorizer et TfidfVectorizer pour les représentations clairsemées.",
    "Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.",
    "FAISS, HNSW (hnswlib), Annoy et ScaNN accélèrent la recherche de plus proches voisins approximatifs (ANN).",
    "Sentence-BERT (SBERT), E5, MPNet et ColBERT sont populaires pour l'indexation dense ou late interaction.",
    "Elasticsearch et OpenSearch implémentent BM25 par défaut et supportent la recherche hybride sparse+dense.",

    # --- Évaluation ---
    "L'évaluation compare précision, rappel, F1, MAP, MRR, nDCG@k et taux de couverture des passages.",
    "MRR mesure l'inverse du rang de la première réponse correcte ; nDCG mesure des gains gradués.",
    "Les jeux de données TREC, MS MARCO et BEIR sont des bancs d'essai courants.",
    "La validation croisée et le split train/dev/test évitent le surapprentissage sur le corpus d'évaluation.",
    "Le jugement de pertinence peut être humain, heuristique ou obtenu via pooling multi-systèmes.",

    # --- Multilingue & domaines ---
    "Les requêtes peuvent être multilingues ; les modèles multilingues mappent plusieurs langues dans le même espace.",
    "Dans le domaine médical, la précision terminologique est cruciale (ex. 'hypertension artérielle').",
    "En droit, les synonymes et références croisées (articles, alinéas) créent de la variance lexicale.",
    "Les recettes de cuisine génèrent des listes d'ingrédients qui favorisent la pondération TF simple.",
    "La recherche académique bénéficie d'expressions multi-mots comme 'régression logistique' ou 'réseau de neurones'.",

    # --- Bruit, longueur et structure ---
    "Certains documents sont très courts: 'BM25 expliqué rapidement'.",
    "D'autres sont longs et contiennent des sections, des listes, des liens et des tableaux qui influencent la tokenisation.",
    "URLs et hashtags (#IR, #NLP) perturbent parfois la segmentation en tokens.",
    "Les citations de code `pip install faiss-gpu` ou `from sklearn.feature_extraction.text import TfidfVectorizer` doivent être préservées.",

    # --- Recherche hybride & reranking ---
    "La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.",
    "Le reranking par un cross-encoder (ex. monoT5, cross-encoder-MiniLM) affine les top-k candidats.",
    "La réduction de dimension (PCA) ou la quantification des vecteurs (PQ, IVFPQ) accélère l'ANN au prix d'une légère perte de rappel.",
    "Le champ 'title' a souvent plus de poids que 'body' dans les moteurs de recherche.",
    "La désambiguïsation des acronymes (ex. 'IR' = 'information retrieval' ou 'imagerie par résonance') dépend du contexte.",

    # --- Cas concrets & exemples mixtes ---
    "Le sac de mots (bag of words) est simple et robuste mais ne capture pas la sémantique profonde.",
    "TF–IDF (avec tiret demi-cadratin) est équivalent à TF-IDF (avec tiret standard) pour la plupart des implémentations.",
    "Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.",
    "Une requête négative comme 'BM25 sans normalisation de longueur' nécessite des opérateurs booléens.",
    "La recherche de vecteurs denses peut tolérer les fautes: 'embeding denses' correspond souvent à 'embeddings denses'.",
    "Vector search peut être sparse or dense depending on the representation (mélange FR/EN).",
    "L'alignement d'espaces multilingues permet 'apprentissage' ≈ 'learning' ≈ 'aprendizaje'.",
    "Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.",
    "Le choix des stop words en français diffère de l'anglais (ex. 'au', 'aux', 'des', 'du').",
    "Le pré-traitement influence fortement les résultats, surtout avec de petites requêtes.",
    "La normalisation Unicode (NFC/NFKD) affecte le traitement des diacritiques (é/ê/e).",
    "En RAG, l'ordre : retrieve → read → generate ; la qualité du retrieve détermine souvent 80% du succès perçu.",
    "Une indexation incrémentale maintient la fraîcheur des résultats sans reconstruire tout l'index."
]

queries = [
    # Requêtes ciblées IR
    "comment fonctionne BM25 (k1, b) pour le ranking ?",
    "différence entre sac de mots et TF-IDF",
    "CountVectorizer vs TfidfVectorizer scikit-learn",
    "bm25 ranking vs dense retrieval",
    "quels avantages des embeddings denses pour la sémantique ?",

    # Prétraitement & robustesse
    "liste de stopwords en français et impact sur la précision",
    "lemmatisation vs stemming pour le français",
    "normalisation des accents et diacritiques",
    "gerer les fautes de frappe en recherche d'information",
    "traiter les hashtags et urls dans les documents",

    # Évaluation
    "comparer MAP, MRR et nDCG@10",
    "datasets d'évaluation: TREC, MS MARCO, BEIR",
    "pooling et jugement de pertinence gradué",
    "quelle métrique pour la pertinence multi-niveaux ?",

    # Multilingue & domaines
    "recherche multilingue avec SBERT ou E5",
    "RAG pour questions médicales",
    "désambiguïser l'acronyme IR selon le contexte",
    "recherche juridique: articles et alinéas",
    "recettes: pondération TF simple vs BM25",

    # ANN & systèmes
    "HNSW vs FAISS vs Annoy pour ANN",
    "hybrid search: combiner BM25 et embeddings",
    "cross-encoder pour reranking des top-k",
    "quantification des vecteurs (IVFPQ) et rappels",
    "pondérer le champ title plus que body",

    # Requêtes bruitées / mixtes / typos
    "tf–idf (variante typographique) explication",
    "embeding denses (typo) utilité",
    "Okapi-BM25 formule",
    "vector search sparse or dense",
    "BM25 sans normalisation de longueur"
]

def show_topk(query: str, doc_ids: np.ndarray, scores: np.ndarray, k: int = 5):
    print(f"\nQuery: {query!r}")
    print("-" * 80)
    order = np.argsort(scores)[::-1][:k]
    for r, i in enumerate(order, 1):
        print(f"[{r:>2}] score={scores[i]:.4f}  doc#{doc_ids[i]} → {corpus[doc_ids[i]]}")



## 1) Génération d'embeddings — notions clés

- **Word2Vec / GloVe** : embeddings **de mots** appris à partir de co-occurrences (statistiques).  
  Pour obtenir un embedding de phrase/document : on peut **moyenner** les vecteurs des mots (simple mais efficace).  
- **BERT & dérivés** : modèles **contextuels** (chaque mot dépend du contexte). Pour des embeddings de *phrase*, on utilise
  des modèles **Sentence-Transformers** spécialement *finetunés* pour la similarité sémantique.



### 1.A — Démo rapide **Word2Vec** (toy) avec `gensim`

On entraîne un mini **Word2Vec** sur notre corpus (ridiculement petit) pour illustrer la mécanique, puis on crée des embeddings de documents par **moyenne** de mots.
> Cela **n'est pas** un modèle de qualité — c'est juste un *exemple pédagogique*.


In [ ]:

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


tokenized = [simple_preprocess(doc) for doc in corpus]


w2v = Word2Vec(sentences=tokenized, vector_size=64, window=5, min_count=1, workers=1, sg=1, epochs=100)


import numpy as np

def doc_embedding_w2v(tokens: List[str]) -> np.ndarray:
    vecs = [w2v.wv[t] for t in tokens if t in w2v.wv]
    if not vecs:
        return np.zeros(w2v.vector_size, dtype=float)
    v = np.mean(vecs, axis=0)
    # Normalisation pour cosinus
    n = np.linalg.norm(v) + 1e-9
    return v / n

doc_vecs_w2v = np.vstack([doc_embedding_w2v(tok) for tok in tokenized])

def search_w2v(query: str, top_k: int = 5):
    q_vec = doc_embedding_w2v(simple_preprocess(query)).reshape(1, -1)
    scores = (doc_vecs_w2v @ q_vec.T).ravel()  # produit scalaire 
    ids = np.arange(len(corpus))
    show_topk(query, ids, scores, k=top_k)

for q in queries:
    search_w2v(q, top_k=5)



Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'
--------------------------------------------------------------------------------
[ 1] score=0.9542  doc#42 → Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 2] score=0.9359  doc#4 → Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.
[ 3] score=0.8778  doc#35 → La recherche hybride combine BM25 pour l'exact match et embeddings denses pour la similarité sémantique.
[ 4] score=0.8769  doc#31 → Certains documents sont très courts: 'BM25 expliqué rapidement'.
[ 5] score=0.8690  doc#47 → Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.

Query: 'différence entre sac de mots et TF-IDF'
--------------------------------------------------------------------------------
[ 1] score=0.9414  doc#17 → Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.
[ 2] score=0.9380


### 1.B — **Embeddings de phrases** avec *Hugging Face* / `sentence-transformers`

On utilise un modèle pré entrainer sur la langue francaise pour bien gérer le français, par exemple :
- `camembert` (rapide, polyvalent).



Le pipeline de base :
1. Charger le modèle d'embeddings.
2. Encoder le corpus → matrice `D × dim`.
3. Encoder la requête et calculer la **similarité cosinus**.


In [4]:


# %pip -q install sentence-transformers

from sentence_transformers import SentenceTransformer, util
import numpy as np


ST_MODEL_NAME = "almanach/camembert-base"
st_model = SentenceTransformer(ST_MODEL_NAME)

# Encodage des documents
doc_vecs_st = st_model.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

def search_st(query: str, top_k: int = 5):
    q_vec = st_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    scores = doc_vecs_st @ q_vec  # produit scalaire = cosinus car normalisé
    ids = np.arange(len(corpus))
    show_topk(query, ids, scores, k=top_k)

for q in queries:
    search_st(q, top_k=5)


No sentence-transformers model found with name almanach/camembert-base. Creating a new one with mean pooling.
c:\Users\salah\anaconda3\envs\ragllm\lib\site-packages\transformers\models\camembert\modeling_camembert.py:388: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(



Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'
--------------------------------------------------------------------------------
[ 1] score=0.9269  doc#42 → Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 2] score=0.9213  doc#4 → Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.
[ 3] score=0.9180  doc#17 → Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.
[ 4] score=0.9060  doc#31 → Certains documents sont très courts: 'BM25 expliqué rapidement'.
[ 5] score=0.9048  doc#47 → Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.

Query: 'différence entre sac de mots et TF-IDF'
--------------------------------------------------------------------------------
[ 1] score=0.9239  doc#17 → Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.
[ 2] score=0.9192  doc#31 → Certains document


## 2) Indexation avec **FAISS** 

Pour une grande collection, on utilise un index de recherche de voisins approchés (ANN).  
**Astuce** : pour la **cosine**, normalisez les vecteurs et utilisez un index `IndexFlatIP` (produit interne). 


In [5]:

# %pip -q install faiss-cpu

import faiss
import numpy as np


dim = doc_vecs_st.shape[1]
index = faiss.IndexFlatIP(dim)  # alternative IndexFlatL2
index.add(doc_vecs_st.astype(np.float32))
print("Index FAISS — nb vecteurs:", index.ntotal)

def faiss_search(query: str, top_k: int = 5):
    q = st_model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    scores, ids = index.search(q, top_k)  #
    print(f"\nQuery: {query!r}")
    print("-" * 80)
    for r, (s, i) in enumerate(zip(scores[0], ids[0]), start=1):
        print(f"[{r:>2}] score={float(s):.4f}  doc#{int(i)} → {corpus[int(i)]}")

# Démo
for q in queries:
    faiss_search(q, top_k=5)


Index FAISS — nb vecteurs: 53

Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'
--------------------------------------------------------------------------------
[ 1] score=0.9269  doc#42 → Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 2] score=0.9213  doc#4 → Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.
[ 3] score=0.9180  doc#17 → Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.
[ 4] score=0.9060  doc#31 → Certains documents sont très courts: 'BM25 expliqué rapidement'.
[ 5] score=0.9048  doc#47 → Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.

Query: 'différence entre sac de mots et TF-IDF'
--------------------------------------------------------------------------------
[ 1] score=0.9239  doc#17 → Gensim facilite le topic modeling (LDA) et des pipelines BoW/TF-IDF rapides.
[ 2] score=0.91


## 3) **Re-ranking** avec un **Cross-Encoder** (HF)

Les embeddings donnent un **premier tri rapide**. Pour un meilleur ordre final, on ré-évalue les **Top-N** candidats
avec un **Cross-Encoder** (un modèle qui lit *la paire* `(requête, document)` et prédit un score de pertinence).

Exemple : `cross-encoder/ms-marco-MiniLM-L-6-v2` (principalement entraîné en anglais, mais fonctionne souvent raisonnablement en FR).  
Pour du 100% FR, choisir un reranker multilingue quand c'est possible.


In [6]:

# %pip -q install sentence-transformers

from sentence_transformers import CrossEncoder

# Reranker (vous pouvez choisir un autre modèle HF)
CE_MODEL_NAME = "antoinelouis/crossencoder-camembert-base-mmarcoFR"
reranker = CrossEncoder(CE_MODEL_NAME)

def rerank_with_cross_encoder(query: str, top_n_from_dense: int = 8, final_k: int = 5):
    # 1) Récupération dense (FAISS) pour réduire la recherche
    q = st_model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype(np.float32)
    scores, ids = index.search(q, top_n_from_dense)
    candidate_ids = ids[0].tolist()

    # 2) Paires (query, passage) pour le CrossEncoder
    pairs = [(query, corpus[i]) for i in candidate_ids]
    ce_scores = reranker.predict(pairs)  # plus lent mais plus précis
    order = np.argsort(-ce_scores)[:final_k]

    print(f"\nQuery: {query!r}  —  Re-ranking sur {top_n_from_dense} candidats")
    print("-" * 80)
    for r, j in enumerate(order, start=1):
        i = candidate_ids[j]
        print(f"[{r:>2}] CE_score={float(ce_scores[j]):.4f}  doc#{i} → {corpus[i]}")

# Démo
for q in queries:
    rerank_with_cross_encoder(q, top_n_from_dense=8, final_k=5)



Query: 'comment fonctionne BM25 (k1, b) pour le ranking ?'  —  Re-ranking sur 8 candidats
--------------------------------------------------------------------------------
[ 1] CE_score=0.9978  doc#4 → Okapi BM25 est un schéma de ranking basé sur la saturation de fréquence et la normalisation par longueur de document.
[ 2] CE_score=0.9968  doc#47 → Les paramètres clés de BM25 sont k1 et b ; k1 contrôle la saturation, b la normalisation de longueur.
[ 3] CE_score=0.9649  doc#42 → Okapi-BM25 et bm25 ranking désignent la même famille de fonctions de scoring.
[ 4] CE_score=0.8686  doc#43 → Une requête négative comme 'BM25 sans normalisation de longueur' nécessite des opérateurs booléens.
[ 5] CE_score=0.0550  doc#31 → Certains documents sont très courts: 'BM25 expliqué rapidement'.

Query: 'différence entre sac de mots et TF-IDF'  —  Re-ranking sur 8 candidats
--------------------------------------------------------------------------------
[ 1] CE_score=0.8537  doc#3 → TF-IDF pondère fort 